In [1]:
%load_ext autoreload
%autoreload 2

In [19]:
import os
import ee
import xee
import geemap
import geopandas as gpd
import numpy as np
import pandas as pd
import xarray
import matplotlib.pyplot as plt
from shapely.geometry import box
from scipy.signal import savgol_filter
import rasterio
from rasterio.features import rasterize
from osgeo import gdal
import rioxarray

from func_mask import get_s2, add_cld_shdw_mask, apply_mask, addNDVI
from func_processing import time_series


In [20]:
try:
    #ee.Initialize()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate(project='ee-katovanpoucke')

# Downloading time series
In this Jupyter notebook, raster time series are downloaded with the Xarray Earth Engine (XEE) package.

## Settings

In [15]:
# Voor twee jaar
start = '2021-11-01' # iets langer dan period of interest zodat we kunnen interpoleren
end = '2024-01-15'

start_wdw = '2022-02-01' # de echte period of interest
end_wdw = '2023-09-01'

vi_str = 'bsi' # welke vegetatie-index
sg_filter = True # gebruik van een Savitzky-Golay filter 

output_folder = 'Q:/Projects/PRJ_MBAG/4d_bwk/project-telcirkels/rasters-telcirkels'
export = True

## Reading geometries (telcirkels)
Processing of the geometries was done in R script 01-process-geometries.R. Outputs were written to shared Q-drive. These processed geometries are now loaded here, so that they can be used in xarray earth engine. The counting points stay the same over the years, this means that we will download the required imagery for all years we want (e.g. 2022 and 2023) at the same time.

In [16]:
#Read telcirkels
#  CRS to 4326!!

tc = gpd.read_file('../data/processed/steekproef_mbag_mas.gpkg')
tc = tc.to_crs(epsg=4326)
tc.head(5)

,pointid,sample_order,batch,regio,area_prop_sb,openheid_klasse,sbp,crs,geometry
0,Km_10015,238,eerste set,Kempen,0.006130,HOL,binnen,31370,"POLYGON ((5.42787 51.11650, 5.42787 51.11636, ..."
1,Km_10019,1,eerste set,Kempen,0.001557,HOL,binnen,31370,"POLYGON ((5.46512 51.10721, 5.46511 51.10707, ..."
2,Km_10056.3,5,eerste set,Kempen,0.000000,HOL,binnen,31370,"POLYGON ((5.42861 51.06523, 5.42860 51.06509, ..."
3,Km_10565.2.3,201,eerste set,Kempen,0.000000,HOL,binnen,31370,"POLYGON ((4.88856 50.97978, 4.88856 50.97964, ..."
4,Km_10577.3,13,eerste set,Kempen,0.000000,HOL,binnen,31370,"POLYGON ((4.87989 50.99397, 4.87988 50.99383, ..."


In [17]:
## testje
#tc = tc.loc[tc.pointid.isin(['Lm_11070','Lm_11568.1','Lm_15064.9', 'Lm_8486.3', 'Lm_8898.1.8', 'Lm_9709.1.6', 'Zn_10111.7', 'Zn_10583.5', 'Zn_11411.7'])]
tc.loc[tc.pointid.isin(['Lm_11070','Lm_11568.1','Lm_15064.9', 'Lm_8486.3', 'Lm_8898.1.8', 'Lm_9709.1.6'])]
tc = tc.reset_index(drop=True)


## Processing for-loop
- Getting time series for every counting point, for all years at once
- Exporting them

In [21]:
# Loop over every telcirkel, for each one, data
sg_name = '_sg' if sg_filter == True else ''

for i in range(tc.shape[0]): #
    # create a bbox and convert to right format for extracting time series
    pointid = tc.loc[i,'pointid']
    shape = tc.loc[i,'geometry']


    # Create time series with custom function
    da = time_series(start, end, start_wdw, end_wdw, shape, vi_arg = vi_str, sg = sg_filter) #input= 'EPSG:4326', output = 'EPSG:32631'
    da.rio.set_spatial_dims('X', 'Y', inplace=True)

    #-- EXPORT
    if export == True:
        # Rename dimensions to conform to rioxarray requirements
        da = da \
        .rename({'Y': 'y', 'X': 'x'}) \
        .transpose('time', 'y', 'x') \
        .rio.write_crs("EPSG:32631")


        # Specify the output file path & export as GTiff

        output_file = f'multiband_raster_{vi_str}_{pointid}{sg_name}.tif'
        output_path = os.path.join(output_folder, output_file)
        da.rio.to_raster(output_path, driver='GTiff', tiled=True, compress='LZW')

        # Band renaming with GDAL (to store the dates)
        gdal.UseExceptions()

        # Define band names (these should be in the order of your bands)
        band_names = [str(time).split('T')[0] for time in da.time.values]

        # Open the dataset in update mode
        dataset = gdal.Open(output_path, gdal.GA_Update)

        if dataset:
            for i, name in enumerate(band_names, start=1):
                band = dataset.GetRasterBand(i)
                if band:
                    band.SetDescription(name)  # Set the band name

            dataset = None  # Close the dataset to flush changes
        else:
            print("Error opening file")

crs roi EPSG:32631
crs roi EPSG:32631
crs roi EPSG:32631


KeyboardInterrupt: 